In [1]:
import random
import os
import shutil
import glob
import gc
import sys
import h5py
import time
import datetime
import pickle
import librosa
import warnings
import matplotlib.pyplot as plt

import numpy as np
from numpy import array
import pandas as pd
from pandas.plotting import parallel_coordinates
from pydub import AudioSegment

#imported for testing
import wave
import contextlib

# for outputing file
from scipy.cluster.vq import vq, kmeans, whiten
import scipy.stats.stats as st

from sklearn.cluster import KMeans
from sklearn.datasets import make_classification
from sklearn.datasets.samples_generator import make_blobs
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize
from sklearn.metrics import auc, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.utils import class_weight

import keras

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.layers import Convolution1D, Dense, MaxPooling1D, Flatten, Add, Dropout, Input, Activation
from keras.layers import TimeDistributed, Bidirectional, LSTM, LeakyReLU
from keras.models import Sequential
from keras import optimizers, regularizers
from keras.utils import np_utils, to_categorical

#from tensorflow.python.client import device_lib
from keras import backend
import tensorflow as tf


C:\Users\Ash Gao\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [2]:
# load training and testing ... 

condition = 'all'

new_train = 'train//' + condition + '//'
new_test = 'test//' + condition + '//'
new_val = 'val//' + condition + '//'

def load_vectors(path):
    files = sorted(os.listdir(path))
    X = np.expand_dims(np.zeros((48, 272)), axis=0)
    y = []
    for npy in files:
        if 'Calm' in npy or 'Other' in npy:
            continue
        
        current = np.load(path+npy)
        X = np.vstack((X, current))
        label = [files.index(npy)]*len(current)       
        y = y + label       
    X = X[1:]
    y = to_categorical(y)    
    print(X.shape)
    print(y.shape)    
    return X, y
    
X_train, y_train = load_vectors(new_train)
try:
    X_val, y_val = load_vectors(new_val)
except:
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=42)
X_test, y_test = load_vectors(new_test)

(34155, 48, 272)
(34155, 4)
(7551, 48, 272)
(7551, 4)


In [4]:
path = 'models//conflict_clf.hdf5'

def mil_squared_error(y_true, y_pred):
    return tf.keras.backend.square(tf.keras.backend.max(y_pred) - tf.keras.backend.max(y_true))

adam = tf.keras.optimizers.Adam(learning_rate=1e-5)

model = tf.keras.models.load_model(path, custom_objects={'mil_squared_error': mil_squared_error})

In [5]:
y_preds = model.predict(X_test)
y_preds = [np.argmax(y) for y in y_preds]
y_trues_temp = [np.argmax(y) for y in y_test]
y_trues = []

for y in y_trues_temp:
    if not y == 1:
        y = 0
        
    y_trues.append(y)

In [6]:
print(len(y_preds))
print(len(y_trues))

7551
7551


In [9]:
accuracy_score(y_trues, y_preds)

0.694609985432393

In [17]:
x_only_anger = []
y_only_anger = []

for x, y in zip(X_test, y_test):
    
    if np.argmax(y) == 1:
        x_only_anger.append(x)
        y_only_anger.append(np.argmax(y))


In [18]:
y_preds_only_anger = model.predict(np.asarray(x_only_anger))
y_preds_only_anger = [np.argmax(y) for y in y_preds_only_anger]

In [23]:
print(len(y_only_anger))

1942


In [22]:
f1_score(y_only_anger, y_preds_only_anger)

0.9431292517006803

In [20]:
accuracy_score(y_only_anger, y_preds_only_anger)

0.892378990731205